In [66]:
from nltk.corpus import brown
from nltk.classify import PositiveNaiveBayesClassifier
import numpy as np
import random

Доступные категории:

In [6]:
brown.categories()

['adventure',
 'belles_lettres',
 'editorial',
 'fiction',
 'government',
 'hobbies',
 'humor',
 'learned',
 'lore',
 'mystery',
 'news',
 'religion',
 'reviews',
 'romance',
 'science_fiction']

# Подготовим данные

Скачаем предложения по темам религия и новости. 

In [93]:
sentences_news = list(brown.sents(categories = ['news']))
sentences = list(brown.sents(categories=['news' ,' religion']))
random.seed(122)
random.shuffle(sentences)

partition_news = int(len(sentences_news) * 0.75)
partition = int(len(sentences) * 0.75)

X_train_news = sentences_news[:partition_news]
X_train = sentences[:partition]

X_test_news = sentences_news[partition_news:]
y_test_news = [True] * len(X_test_news)

X_test = sentences[partition:]
y_test = [sentence in sentences_news for sentence in X_test]

# Запуск наивного байеса

Напишем функции для построения сета признаков, подсчета числа ошибок.

In [96]:
def features(sentence):
    return dict(('contains(%s)' % word, True) for word in sentence)

def calc_error(prediction, test):
    return np.sum(np.array(prediction) != np.array(test)) / len(prediction)

Для начала обучим классификатор на 3/4 новостных предложений. На оставшейся трети посчитаем число ошибок, а также посчитаем число ошибок на тестовой четверти от всех предложений.

In [97]:
positive_featuresets = list(map(features, X_train_news))
unlabeled_featuresets = list(map(features, X_train))
classifier = PositiveNaiveBayesClassifier.train(positive_featuresets,\
                                                unlabeled_featuresets)

prediction_news = classifier.classify_many(list(map(features, X_test_news)))
error_news = calc_error(prediction, y_test_news)
print('Ошибка классификатора на тестовой части новостных предложений: {}'.format(error_news))

prediction_all = classifier.classify_many(list(map(features, X_test)))
error_all = calc_error(prediction_all, y_test)
print('Ошибка классификатора на тестовой части всех предложений: {}'.format(error_all))

Ошибка классификатора на тестовой части новостных предложений: 0.27076124567474047
Ошибка классификатора на тестовой части всех предложений: 0.19896193771626297


Классификатор не распознает примерно треть от тестовых новостных предложений. Среди всех предложений получилась ошибка около 20%.

Обучим теперь классификатор на всех доступных новостных предложениях и посчитаем ошибку на тестовой выборке.

In [101]:
positive_featuresets2 = list(map(features, sentences_news))
classifier2 = PositiveNaiveBayesClassifier.train(positive_featuresets2,\
                                                unlabeled_featuresets)

prediction_all2 = classifier2.classify_many(list(map(features, X_test)))
error_all2 = calc_error(prediction_all2, y_test)
print('Ошибка классификатора на тестовой части всех предложений: {}'.format(error_all2))

Ошибка классификатора на тестовой части всех предложений: 0.01124567474048443


Ошибка упала до 11% -- весьма неплохой результат. Поиграемся с классификатором на нескольких выдуманных предложениях.

Ставим вопрос: относится ли следующее предложение к теме новостей?

In [103]:
classifier.classify(features('oh my God!'))

False

In [135]:
classifier.classify(features('The President of Russia today met with Ambassador of Germany.'))

True

In [111]:
classifier.classify(features('Russia.'))

True

In [134]:
classifier.classify(features('The President today met with Ambassador of Germany.'))

True

In [113]:
classifier.classify(features('Ambassador of Germany.'))

True

In [114]:
classifier.classify(features('President today met.'))

True

In [115]:
classifier.classify(features('I\'m playing with my homework on machine learning.'))

False

In [119]:
classifier.classify(features('Lol, what?'))

True

In [126]:
sentence = 'The God today met with Ambassador of Germany.'
classifier.classify(features(sentence))

True

In [131]:
sentence4 =  ', blessed her and remember that she have to read Bible.'
sentence2 = sentence[:-1] + sentence4
classifier.classify(features(sentence2))

True

In [133]:
classifier.classify(features('The God today met with her ' + sentence4))

False